In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 6.4 Congestion games

## 6.4.1 Definition

Congestion games are where agents pay a cost for their actions depending on how many other people take the same action. We have a set of resources, and a cost function. The actions are whether or not to claim the resource. The cost is the same across all agents and only depends on the total number of agents claiming the same resource. 

This kind of game appears quite often. As the name suggests one obvious case is when there are two or more different routes from A to B which both take different amounts of time depending on the number of people who choose each. E.g., a highway vs a backroad. But other cases are also common, for instance choosing which bar to go to. In that case there is a cost if there are too may people or too few people (the cost function can be positive!). There seem to be different definitions flying around and in some (like the example above) each actor may only choose 1 resource, but in general there aren't limits. 

## 6.4.1 Computing equilibria

Congestion games have a pure-strategy equilibrium. Not only that, but the equilibrium can be found simply by iteratively updating each player's response for each action. Proof of this is incoming, but first a demonstration:

Say we have two resources A and B. The cost is linear increasing with the number of people:

In [90]:
def MyopicBestResponse(cost_funcs, n_players):
    num_choices = len(cost_funcs)
    strategies = np.random.randint(0,2,(n_players, num_choices))
    any_change = True
    while any_change:
        any_change = False
        for player in range(n_players):
            sum_others = np.sum(strategies,axis=0)-strategies[player]
            for choice in range(num_choices):
                cost = cost_funcs[choice](sum_others[choice]+1)
                if cost>0 and strategies[player,choice]==1:
                    any_change = True
                    strategies[player,choice] = 0
                if cost<0 and strategies[player,choice]==0:
                    any_change = True
                    strategies[player,choice] = 1
    print(strategies)

costA = lambda x: -3 + 2*x
costB = lambda x: -1.5 + x
print("example equilibria")
MyopicBestResponse([costA,costB], 2)

example equilibria
[[1 1]
 [0 0]]


In this case the equilibria are:

$$
\begin{bmatrix}
1 & 1 \\
0 & 0
\end{bmatrix}
\text{or}
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
\text{or}
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
\text{or}
\begin{bmatrix}
0 & 0 \\
1 & 1
\end{bmatrix}
$$

I.e., each resource gets 1 person. It gets more interesting with more people.

In [109]:
costA = lambda x: -5 + x
costB = lambda x: -20 + x**2
MyopicBestResponse([costA,costB], 10)

[[0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 1]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [1 0]]


Now obviously the idea of MyopicBestResponse is not going to work for all normal-form games. E.g., consider the matching pennies problem. 